Projekt MNwS - Aleksander Folfas 411052

In [3]:
import numpy as np
from scipy.stats import kstest
import matplotlib.pyplot as plt

Zadanie 10



Za pomocą odpowiednich symulacji zbadać wpływ liczby klas (na które dzielone są dane) odsetek
odrzuceń prawdziwej hipotezy głównej (tzn. błąd I rodzaju) w teście 𝜒
2 w przypadku weryfikacji
zgodności z rozkładem 𝑡-Studenta o różnych stopniach swobody, tzn. generować dane pochodzące z
rozkładu 𝑡-Studenta o 𝑘 stopniach swobody i weryfikować hipotezę, że dane mają rozkład 𝑡-Studenta
o 𝑘 st. swobody. Uzyskane wyniki należy przedstawić na odpowiednich wykresach ilustrujących
rozmiary testów z uwzględnieniem:
- liczby danych,
- liczby klas,
- liczby stopni swobody generowanego rozkładu